## Can we get data near the [Coastal Endurance](http://oceanobservatories.org/array/coastal-endurance/) array?

(a) Time span to get the data (&plusmn;4 days).

In [ ]:
from datetime import datetime, timedelta

now = datetime.utcnow()
start = now - timedelta(days=4)
stop = now + timedelta(days=4)

(b) Bounding box: everything inside the Oregon Line (44&deg;35'N, 125&deg;W) and Washington Line (47&deg;N, 125&deg;W).

In [ ]:
bbox = [-125, 44.583, -123.75, 47]

(c) Variable: sea water temperature.

In [ ]:
from utilities import CF_names

sos_name = 'sea_water_temperature'
name_list = CF_names[sos_name]

Assemble `fes` filter with `a`+`b`+`c`.

In [ ]:
from owslib import fes
from utilities import fes_date_filter

kw = dict(wildCard='*',
          escapeChar='\\',
          singleChar='?',
          propertyname='apiso:AnyText')

or_filt = fes.Or([fes.PropertyIsLike(literal=('*%s*' % val), **kw)
                  for val in name_list])

begin, end = fes_date_filter(start, stop)
filter_list = [fes.And([begin, end, fes.BBox(bbox), or_filt])]

Instantiate `csw` object using the NGDC catalog.

In [ ]:
from owslib.csw import CatalogueServiceWeb

csw = CatalogueServiceWeb('http://www.ngdc.noaa.gov/geoportal/csw',
                          timeout=60)

csw.getrecords2(constraints=filter_list, maxrecords=1000, esn='full')

fmt = '{:*^64}'.format
print(fmt(' Catalog information '))
print("CSW version: {}".format(csw.version))
print("Number of datasets available: {}".format(len(csw.records.keys())))

What did we get?

In [ ]:
from utilities import service_urls

dap_urls = service_urls(csw.records, service='odp:url')
sos_urls = service_urls(csw.records, service='sos:url')

print(fmt(' SOS '))
for url in sos_urls:
    print('{}'.format(url))

print(fmt(' DAP '))
for url in dap_urls:
    print('{}.html'.format(url))

We found [`CO-OPS`](http://opendap.co-ops.nos.noaa.gov/) and [`NDBC`](http://www.ndbc.noaa.gov/) observations available in the [`SOS`](http://www.opengeospatial.org/standards/sos) schema and one model available in OPeNDAP.

OPeNDAP URLs can be fed directly to any OPenDAP client,
but there is a gap between finding the SOS endpoint and actually downloading the data.
We can bridge that gap using `pyoos`'s collectors (`NdbcSos` and `CoopsSos`).

PS: We should automate this step.  (See https://github.com/SECOORA/secoora/issues/232.)

Get NDBC.

In [ ]:
from pyoos.collectors.ndbc.ndbc_sos import NdbcSos

collector_ndbc = NdbcSos()

collector_ndbc.set_bbox(bbox)
collector_ndbc.end_time = stop
collector_ndbc.start_time = start
collector_ndbc.variables = [sos_name]

ofrs = collector_ndbc.server.offerings
title = collector_ndbc.server.identification.title
print(fmt(' NDBC Collector offerings '))
print('{}: {} offerings'.format(title, len(ofrs)))

In [ ]:
from pandas import DataFrame
from owslib.ows import ExceptionReport
from utilities import collector2table, get_ndbc_longname

try:
    ndbc = collector2table(collector=collector_ndbc)

    names = []
    for s in ndbc['station']:
        try:
            name = get_ndbc_longname(s)
        except ValueError:
            name = s
        names.append(name)

    ndbc['name'] = names

    ndbc.set_index('name', inplace=True)
except ExceptionReport:
    ndbc = DataFrame()

ndbc

Get COOPS.

In [ ]:
from pyoos.collectors.coops.coops_sos import CoopsSos

collector_coops = CoopsSos()

collector_coops.set_bbox(bbox)
collector_coops.end_time = stop
collector_coops.start_time = start
collector_coops.variables = [sos_name]

ofrs = collector_coops.server.offerings
title = collector_coops.server.identification.title
print(fmt(' Collector offerings '))
print('{}: {} offerings'.format(title, len(ofrs)))

In [ ]:
from utilities import get_coops_metadata

try:
    coops = collector2table(collector=collector_coops)

    names = []
    for s in coops['station']:
        try:
            name = get_coops_metadata(s)[0]
        except ValueError:
            name = s
        names.append(name)

    coops['name'] = names

    coops.set_index('name', inplace=True)
except ExceptionReport:
    coops = DataFrame()

coops

Now we can merge everything that we found into one table before we start downloading the data.

In [ ]:
from pandas import concat

all_obs = concat([coops, ndbc])

all_obs

The custom function `pyoos2df` uses the `pyoos` collector to download each observation found as a `pandas.DataFrame`.
We accumulate all the DataFrames into the dictionary `data`.

In [ ]:
from utilities import pyoos2df

data = dict()
col = 'sea_water_temperature (C)'
for station in all_obs.index:
    try:
        idx = all_obs['station'][station]
        df = pyoos2df(collector_ndbc, idx, df_name=station)
        if df.empty:
            df = pyoos2df(collector_coops, idx, df_name=station)
        data.update({idx: df[col]})
    except ExceptionReport as e:
        print("[{}] {}:\n{}".format(idx, station, e))

Let's take a look at the data we found.

In [ ]:
%matplotlib inline

import seaborn
import matplotlib.pyplot as plt
fig, ax = plt.subplots(figsize=(13, 3.25))

colors = seaborn.color_palette("Set2", len(data))
for k, (key, series) in enumerate(data.items()):
    station_name = all_obs.index[all_obs['station'] == key][0]
    series.plot(ax=ax, label=station_name, color=colors[k])
    ax.legend(bbox_to_anchor=(1.35, 1))

```python
from utilities import is_station

non_stations = []
for url in dap_urls:
    try:
        if not is_station(url):
            non_stations.append(url)
    except RuntimeError as e:
        print("Could not access URL {}. {!r}".format(url, e))

dap_urls = non_stations

print(fmt(' Filtered DAP '))
for url in dap_urls:
    print('{}.html'.format(url))
```

```python
from utilities import quick_load_cubes, get_surface, get_model_name

url = dap_urls[0]

cube = quick_load_cubes(url, name_list, callback=None, strict=True)
cube = get_surface(cube)
mod_name, model_full_name = get_model_name(cube, url)
```

Since we found only one model let's just its mesh.
For that we will need to use `pyugrid` (this is a UGRID compliant model).

PS: I use matplotlib for the triangulation because it is faster than `pyugrid`.

In [ ]:
import pyugrid
import matplotlib.tri as tri

ugrid = pyugrid.UGrid.from_ncfile(url)
lon = ugrid.nodes[:, 0]
lat = ugrid.nodes[:, 1]
triangles = ugrid.faces[:]
triang = tri.Triangulation(lon, lat, triangles=triangles)

The next two cells creates a PNG image with the mesh and saves a trimmed version that we can use as a map layer.

Note that we could use more geo-friendly vector formats, like GeoJSON, but due to the large numbers of elements in the mesh it is lighter to use a raster overlay instead.

In [ ]:
import cartopy.crs as ccrs
import cartopy.feature as cfeature

bounds = lon.min(), lon.max(), lat.min(), lat.max()

def make_map(bbox, projection=ccrs.Mercator()):
    fig, ax = plt.subplots(subplot_kw=dict(projection=projection))
    ax.set_extent(bounds)
    return fig, ax

fig, ax = make_map(bbox, projection=ccrs.PlateCarree())
kw = dict(linestyle='-', color='darkgray', linewidth=0.5)
ax.triplot(triang, **kw)
ax.axis('off')

fig.savefig('mesh.png', transparent=True, dpi=150)

In [ ]:
import numpy as np
from PIL import Image, ImageChops


def trim(fname):
    img = Image.open(fname)
    img = img.convert('RGBA')
    datas = img.getdata()

    new_data = []
    for item in datas:
        if item[0] == 255 and item[1] == 255 and item[2] == 255:
            new_data.append((255, 255, 255, 0))
        else:
            new_data.append(item)
    img.putdata(new_data)
    
    border = Image.new(img.mode, img.size, img.getpixel((0, 0)))
    diff = ImageChops.difference(img, border)
    diff = ImageChops.add(diff, diff, 2.0, -100)
    bbox = diff.getbbox()
    if bbox:
        img = img.crop(bbox)
    return np.array(img)

img = trim('mesh.png')

Now that we have all the pieces let's create an interactive map with all this information.

First the `Map` object.

In [ ]:
import folium
import numpy as np

location = np.array(bbox).reshape(2, 2).mean(axis=0).tolist()[::-1]
tiles = ('http://services.arcgisonline.com/arcgis/rest/'
         'services/Ocean/World_Ocean_Base/MapServer/tile/{z}/{y}/{x}')

mapa = folium.Map(location=location, zoom_start=7, tiles=tiles, attr='ESRI')

The time-series (as html `bokeh` plots).

In [ ]:
from bokeh.plotting import figure
from bokeh.resources import CDN
from bokeh.embed import file_html

from folium.element import IFrame

def make_marker(key):
    width, height = 500, 250
    station_name = all_obs.index[all_obs['station'] == key][0]
    
    p = figure(x_axis_type="datetime",
               title=station_name,
               width=width, height=height)
    p.line(data[key].index, data[key], line_width=2)
    html = file_html(p, CDN, key)
    iframe = IFrame(html, width=width+40, height=height+80)
    
    pos = (all_obs[all_obs['station'] == key]['lat'][0],
           all_obs[all_obs['station'] == key]['lon'][0])
    
    popup = folium.Popup(iframe, max_width=2650)
    icon = folium.Icon(color='green', icon='stats')
    marker = folium.Marker(pos, popup=popup, icon=icon)
    return marker

for key in data.keys():
    make_marker(key).add_to(mapa)

A region bounding box.

In [ ]:
box = folium.PolyLine([[bbox[1], bbox[0]], [bbox[1], bbox[2]],
                       [bbox[3], bbox[2]], [bbox[3], bbox[0]],
                       [bbox[1], bbox[0]]], color='red')
box.add_to(mapa)

The model mesh (raster layer).

In [ ]:
from netCDF4 import Dataset
from folium import plugins

with Dataset(url) as nc:
    title = nc.title

mesh = plugins.ImageOverlay(img,
                            bounds=[[lat.min(), lon.min()],
                                    [lat.max(), lon.max()]],
                            opacity=0.5)

folium.Popup(title).add_to(mesh)
mesh.add_to(mapa)

Finally the full map:

In [ ]:
mapa

And the original Endurance Array image.

![](http://oceanobservatories.org/wp-content/uploads/2011/04/Endurance-Array-Map_2013_04-17_ver_0-02.jpg)